<a href="https://colab.research.google.com/github/ImagingDataCommons/Cloud-Resources-Workflows/blob/config_fix/Notebooks/Totalsegmentator/dicomSEGMaps/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Importing Packages**

In [1]:
!pip install natsort

In [2]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time
import ast
from natsort import natsorted

###**Runtime Environment**

In [3]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Wed Oct 04 09:48:30 2023

Current directory :/content
Python version    : 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


###**Download TotalSegmentator mappings from Slicer Documentation**

In [4]:
try:
  os.remove(f'{curr_dir}/TotalSegmentator.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/vkt1414/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py

--2023-10-04 09:48:32--  https://raw.githubusercontent.com/vkt1414/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63666 (62K) [text/plain]
Saving to: ‘TotalSegmentator.py’

TotalSegmentator.py 100%[===================>]  62.17K  --.-KB/s    in 0.01s   

2023-10-04 09:48:32 (4.46 MB/s) - ‘TotalSegmentator.py’ saved [63666/63666]



###**Data Wrangling**

In [5]:
try:
  os.remove(f'{curr_dir}/map_to_binary.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/wasserth/TotalSegmentator/master/totalsegmentator/map_to_binary.py
import map_to_binary
label_id_body_part_data = map_to_binary.class_map['total_v1']
label_id_body_part_data_df = pd.DataFrame(list(label_id_body_part_data.items()), columns=['labelID', 'Structure'])
label_id_body_part_data_df

--2023-10-04 09:48:35--  https://raw.githubusercontent.com/wasserth/TotalSegmentator/master/totalsegmentator/map_to_binary.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12965 (13K) [text/plain]
Saving to: ‘map_to_binary.py’

map_to_binary.py    100%[===================>]  12.66K  --.-KB/s    in 0s      

2023-10-04 09:48:36 (90.0 MB/s) - ‘map_to_binary.py’ saved [12965/12965]



,labelID,Structure
0,1,spleen
1,2,kidney_right
2,3,kidney_left
3,4,gallbladder
4,5,liver
...,...,...
99,100,autochthon_left
100,101,autochthon_right
101,102,iliopsoas_left
102,103,iliopsoas_right


In [6]:
with open('TotalSegmentator.py', 'r') as f:
    content = f.read()
    tree = ast.parse(content)

class TotalSegmentatorLogicFinder(ast.NodeVisitor):
    def __init__(self):
        self.result = None

    def visit_ClassDef(self, node):
        if node.name == 'TotalSegmentatorLogic':
            for body_node in node.body:
                if isinstance(body_node, ast.FunctionDef) and body_node.name == '__init__':
                    for expr in body_node.body:
                        if isinstance(expr, ast.Assign):
                            target = expr.targets[0]
                            if isinstance(target, ast.Attribute) and target.attr == 'totalSegmentatorLabelTerminology':
                                self.result = ast.literal_eval(ast.unparse(expr.value))
        self.generic_visit(node)

finder = TotalSegmentatorLogicFinder()
finder.visit(tree)
totalSegmentatorLabelTerminology = finder.result
data = []
for key, value in totalSegmentatorLabelTerminology.items():
    fields = value.split('~')
    row = {'Structure': key}
    for i, field in enumerate(fields):
        subfields = field.split('^')
        if i == 0:
            if len(subfields) > 1:
                row['SegmentAlgorithmName'] = subfields[0]
                row['SegmentAlgorithmType'] = subfields[1]
        elif i == 1:
            if len(subfields) > 2:
                row['SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyCategoryCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyCategoryCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 2:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 3:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'] = subfields[2]
    data.append(row)

df = pd.DataFrame(data)

from natsort import natsort_keygen

df = df.sort_values('Structure', key=natsort_keygen()).reset_index(drop=True)

slicer_merged_df = pd.merge(label_id_body_part_data_df,df, left_on='Structure', right_on='Structure')
slicer_merged_df

,labelID,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,1,spleen,SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,
1,2,kidney_right,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right
2,3,kidney_left,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left
3,4,gallbladder,SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,
4,5,liver,SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,
...,...,...,...,...,...,...,...,...,...,...,...
99,100,autochthon_left,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,7771000,Left
100,101,autochthon_right,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,24028007,Right
101,102,iliopsoas_left,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left
102,103,iliopsoas_right,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right


###**MhubAI's dcmqi json config as a template**

In [7]:
try:
  os.remove(f'{curr_dir}/dicomseg_metadata_whole.json')
except OSError:
  pass
!wget https://raw.githubusercontent.com/vkt1414/models/main/models/totalsegmentator/config/dicomseg_metadata_whole.json

with open(f'{curr_dir}/dicomseg_metadata_whole.json', 'r') as file:
    json_data = json.load(file)

mhubai = pd.json_normalize(json_data['segmentAttributes'][0]).reset_index(drop=True)
mhubai=mhubai[['SegmentAlgorithmName', 'SegmentAlgorithmType', 'SegmentDescription',
       'labelID', 'recommendedDisplayRGBValue',]]
mhubai

--2023-10-04 09:48:43--  https://raw.githubusercontent.com/vkt1414/models/main/models/totalsegmentator/config/dicomseg_metadata_whole.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80391 (79K) [text/plain]
Saving to: ‘dicomseg_metadata_whole.json’

dicomseg_metadata_w 100%[===================>]  78.51K  --.-KB/s    in 0.01s   

2023-10-04 09:48:43 (6.83 MB/s) - ‘dicomseg_metadata_whole.json’ saved [80391/80391]



,SegmentAlgorithmName,SegmentAlgorithmType,SegmentDescription,labelID,recommendedDisplayRGBValue
0,TotalSegmentator,AUTOMATIC,Left adrenal gland,12,"[199, 27, 27]"
1,TotalSegmentator,AUTOMATIC,Right adrenal gland,11,"[199, 27, 27]"
2,TotalSegmentator,AUTOMATIC,Aorta,7,"[224, 97, 76]"
3,TotalSegmentator,AUTOMATIC,Left autochthonous back muscle,100,"[192, 104, 88]"
4,TotalSegmentator,AUTOMATIC,Right autochthonous back muscle,101,"[192, 104, 88]"
...,...,...,...,...,...
99,TotalSegmentator,AUTOMATIC,Vertebrae T5,30,"[241, 214, 145]"
100,TotalSegmentator,AUTOMATIC,Vertebrae T6,29,"[241, 214, 145]"
101,TotalSegmentator,AUTOMATIC,Vertebrae T7,28,"[241, 214, 145]"
102,TotalSegmentator,AUTOMATIC,Vertebrae T8,27,"[241, 214, 145]"


###**Generating Updated dcmqi json config using Slicer Mappings**

In [8]:
merged_df = pd.merge(mhubai,slicer_merged_df, left_on='labelID', right_on='labelID')
merged_df = merged_df.drop('Structure', axis=1)
merged_df['SegmentAlgorithmName']='TotalSegmentator v1.5.5'
merged_df=merged_df.sort_values('labelID')
merged_df

,SegmentAlgorithmName,SegmentAlgorithmType,SegmentDescription,labelID,recommendedDisplayRGBValue,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
76,TotalSegmentator v1.5.5,AUTOMATIC,Spleen,1,"[157, 108, 162]",SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,
38,TotalSegmentator v1.5.5,AUTOMATIC,Right kidney,2,"[212, 126, 151]",SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right
37,TotalSegmentator v1.5.5,AUTOMATIC,Left kidney,3,"[212, 126, 151]",SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left
14,TotalSegmentator v1.5.5,AUTOMATIC,Gallbladder,4,"[139, 150, 98]",SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,
39,TotalSegmentator v1.5.5,AUTOMATIC,Liver,5,"[221, 130, 101]",SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,TotalSegmentator v1.5.5,AUTOMATIC,Left autochthonous back muscle,100,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,7771000,Left
4,TotalSegmentator v1.5.5,AUTOMATIC,Right autochthonous back muscle,101,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,24028007,Right
34,TotalSegmentator v1.5.5,AUTOMATIC,Left iliopsoas,102,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left
35,TotalSegmentator v1.5.5,AUTOMATIC,Right iliopsoas,103,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right


In [9]:
try:
  os.remove(f'{curr_dir}/rgb_colors_TotalSegmentator.csv')
except OSError:
  pass
!wget https://raw.githubusercontent.com/ImagingDataCommons/Cloud-Resources-Workflows/config_fix/configs/TotalSegmentator/rgb_colors_TotalSegmentator.csv

colors=pd.read_csv(f'{curr_dir}/rgb_colors_TotalSegmentator.csv')

colors

--2023-10-04 09:48:51--  https://raw.githubusercontent.com/ImagingDataCommons/Cloud-Resources-Workflows/config_fix/configs/TotalSegmentator/rgb_colors_TotalSegmentator.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1550 (1.5K) [text/plain]
Saving to: ‘rgb_colors_TotalSegmentator.csv’

rgb_colors_TotalSeg 100%[===================>]   1.51K  --.-KB/s    in 0s      

2023-10-04 09:48:51 (27.4 MB/s) - ‘rgb_colors_TotalSegmentator.csv’ saved [1550/1550]



,SegmentDescription,recommendedDisplayRGBValue
0,Vertebrae C1,"[255, 186, 107]"
1,Vertebrae C2,"[193, 99, 36]"
2,Vertebrae C3,"[167, 117, 152]"
3,Vertebrae C4,"[250, 14, 116]"
4,Vertebrae C5,"[118, 40, 9]"
5,Vertebrae C6,"[253, 134, 125]"
6,Vertebrae C7,"[255, 0, 0]"
7,Vertebrae T1,"[0, 2, 145]"
8,Vertebrae T2,"[141, 186, 216]"
9,Vertebrae T3,"[30, 128, 188]"


In [11]:
# Set 'SegmentDescription' as the index for both dataframes for the update operation
merged_df.set_index('SegmentDescription', inplace=True)
colors.set_index('SegmentDescription', inplace=True)

# Update 'recommendedDisplayRGBValue' in merged_df with the values from colors
merged_df['recommendedDisplayRGBValue'].update(colors['recommendedDisplayRGBValue'])

# Reset the index
merged_df.reset_index(inplace=True)
colors.reset_index(inplace=True)

# Convert 'recommendedDisplayRGBValue' to list
merged_df['recommendedDisplayRGBValue'] = merged_df['recommendedDisplayRGBValue'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Now write merged_df to JSON
merged_df


,SegmentDescription,SegmentAlgorithmName,SegmentAlgorithmType,labelID,recommendedDisplayRGBValue,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,Spleen,TotalSegmentator v1.5.5,AUTOMATIC,1,"[157, 108, 162]",SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,
1,Right kidney,TotalSegmentator v1.5.5,AUTOMATIC,2,"[212, 126, 151]",SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right
2,Left kidney,TotalSegmentator v1.5.5,AUTOMATIC,3,"[212, 126, 151]",SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left
3,Gallbladder,TotalSegmentator v1.5.5,AUTOMATIC,4,"[139, 150, 98]",SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,
4,Liver,TotalSegmentator v1.5.5,AUTOMATIC,5,"[221, 130, 101]",SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Left autochthonous back muscle,TotalSegmentator v1.5.5,AUTOMATIC,100,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,7771000,Left
100,Right autochthonous back muscle,TotalSegmentator v1.5.5,AUTOMATIC,101,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,24028007,Right
101,Left iliopsoas,TotalSegmentator v1.5.5,AUTOMATIC,102,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left
102,Right iliopsoas,TotalSegmentator v1.5.5,AUTOMATIC,103,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right


In [12]:
merged_df_json = json.loads(merged_df.to_json(orient='records'))

for segment in merged_df_json:
    segment['SegmentedPropertyCategoryCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeMeaning')
    }
    segment['SegmentedPropertyTypeCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyTypeCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyTypeCodeSequence.CodeMeaning')
    }
    if ('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator' in segment and
        (segment['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeValue'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'].strip())):
        segment['SegmentedPropertyTypeModifierCodeSequence'] = {
            'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'),
            'CodeValue': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeValue'),
            'CodeMeaning': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeMeaning')
        }
    else:
        for key in ['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning']:
            if key in segment:
                del segment[key]
final_json = {
    "BodyPartExamined": "CHEST",
    "ClinicalTrialCoordinatingCenterName": "dcmqi",
    "ClinicalTrialSeriesID": "0",
    "ClinicalTrialTimePointID": "1",
    "ContentCreatorName": "IDC",
    "ContentDescription": "Image segmentation",
    "ContentLabel": "SEGMENTATION",
    "InstanceNumber": "1",
    "SeriesDescription": "TotalSegmentator Segmentation",
    "SeriesNumber": "42",
    "segmentAttributes": [merged_df_json]
}
with open('dicomseg_metadata_whole_slicerAsRef.json', 'w') as file:
    json.dump(final_json, file, indent=4)
final_json

{'BodyPartExamined': 'CHEST',
 'ClinicalTrialCoordinatingCenterName': 'dcmqi',
 'ClinicalTrialSeriesID': '0',
 'ClinicalTrialTimePointID': '1',
 'ContentCreatorName': 'IDC',
 'ContentDescription': 'Image segmentation',
 'ContentLabel': 'SEGMENTATION',
 'InstanceNumber': '1',
 'SeriesDescription': 'TotalSegmentator Segmentation',
 'SeriesNumber': '42',
 'segmentAttributes': [[{'SegmentDescription': 'Spleen',
    'SegmentAlgorithmName': 'TotalSegmentator v1.5.5',
    'SegmentAlgorithmType': 'AUTOMATIC',
    'labelID': 1,
    'recommendedDisplayRGBValue': [157, 108, 162],
    'SegmentedPropertyCategoryCodeSequence': {'CodingSchemeDesignator': 'SCT',
     'CodeValue': '123037004',
     'CodeMeaning': 'Anatomical Structure'},
    'SegmentedPropertyTypeCodeSequence': {'CodingSchemeDesignator': 'SCT',
     'CodeValue': '78961009',
     'CodeMeaning': 'Spleen'}},
   {'SegmentDescription': 'Right kidney',
    'SegmentAlgorithmName': 'TotalSegmentator v1.5.5',
    'SegmentAlgorithmType': 'AUTOMA

In [13]:
import yaml
with open('dicomseg_metadata_whole_slicerAsRef.yaml', 'w') as file:
    yaml.dump(final_json, file)


###**Verifying Slicer Mappings against Snomed CT mappings**

In [15]:
import requests
import pandas as pd

base_url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/MAIN'
headers = { 'User-Agent': 'Python'}

# Replace this with the actual language reference set id for your desired language
language_refset = '900000000000509007'

# Replace these with the actual column names from your dataframe
sct_codes = ['SegmentedPropertyCategoryCodeSequence.CodeValue', 'SegmentedPropertyTypeCodeSequence.CodeValue', 'SegmentedPropertyTypeModifierCodeSequence.CodeValue']

for sct_code_column in sct_codes:
    # Create new columns for preferred and acceptable terms
    preferred_terms_column = sct_code_column + '.PreferredTerms'
    acceptable_terms_column = sct_code_column + '.AcceptableTerms'
    df[preferred_terms_column] = None
    df[acceptable_terms_column] = None

    # Get unique SCT codes in column
    unique_sct_codes = df[sct_code_column].unique()

    # Get preferred and acceptable terms for each unique SCT code
    sct_code_terms = {}
    for sct_code in unique_sct_codes:
        if pd.isnull(sct_code):
            # If SCT code is missing, set preferred and acceptable terms to empty arrays
            sct_code_terms[sct_code] = ([], [])
        else:
            # Get all descriptions for the given SCT code
            response = requests.get(f'{base_url}/descriptions?conceptId={sct_code}&limit=50', headers=headers)
            descriptions = response.json()['items']

            # Filter descriptions by acceptability and activity
            preferred_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'PREFERRED']
            acceptable_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'ACCEPTABLE']

            # Store terms for SCT code
            sct_code_terms[sct_code] = (preferred_terms, acceptable_terms)

    # Add arrays to dataframe
    for index, row in df.iterrows():
        sct_code = row[sct_code_column]
        preferred_terms, acceptable_terms = sct_code_terms[sct_code]
        df.at[index, preferred_terms_column] = preferred_terms
        df.at[index, acceptable_terms_column] = acceptable_terms

sct_code_column = 'SegmentedPropertyTypeModifierCodeSequence.CodeValue'
preferred_terms_column = sct_code_column + '.PreferredTerms'
acceptable_terms_column = sct_code_column + '.AcceptableTerms'

# Remove or replace unexpected values in the preferred terms column
for index, row in df.iterrows():
    sct_code = row[sct_code_column]
    if sct_code == '':
        # If SCT code is an empty string, set preferred and acceptable terms to empty arrays
        df.at[index, preferred_terms_column] = []
        df.at[index, acceptable_terms_column] = []

column_order = ['Structure',
                'SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyCategoryCodeSequence.CodeValue',
                'SegmentedPropertyCategoryCodeSequence.CodeMeaning',
                'SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms',
                'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyTypeCodeSequence.CodeValue',
                'SegmentedPropertyTypeCodeSequence.CodeMeaning',
                'SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms',
                'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms']

new_column_names = {'SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTAcceptableTerms',
                    'SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms'}

# Rename the columns of the DataFrame
df = df.rename(columns=new_column_names)
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,...,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms
0,adrenal_gland_left,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,7771000,Left,...,"[Adrenal structure (body structure), Adrenal s...",[Adrenal gland],"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]","[Body structure (body structure), Body structure]",[Body structures],"[Adrenal structure (body structure), Adrenal s...",[Adrenal gland],"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
1,adrenal_gland_right,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,24028007,Right,...,"[Adrenal structure (body structure), Adrenal s...",[Adrenal gland],"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]","[Body structure (body structure), Body structure]",[Body structures],"[Adrenal structure (body structure), Adrenal s...",[Adrenal gland],"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"
2,aorta,SCT,123037004,Anatomical Structure,SCT,15825003,Aorta,,,,...,"[Aortic structure, Aortic structure (body stru...",[Aorta],[],[],"[Body structure (body structure), Body structure]",[Body structures],"[Aortic structure, Aortic structure (body stru...",[Aorta],[],[]
3,autochthon_left,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,7771000,Left,...,[Structure of deep muscle of back (body struct...,[Structure of deep muscle of back],"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]","[Body structure (body structure), Body structure]",[Body structures],[Structure of deep muscle of back (body struct...,[Structure of deep muscle of back],"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
4,autochthon_right,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,24028007,Right,...,[Structure of deep muscle of back (body struct...,[Structure of deep muscle of back],"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]","[Body structure (body structure), Body structure]",[Body structures],[Structure of deep muscle of back (body struct...,[Structure of deep muscle of back],"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,vertebrae_T8,SCT,123037004,Anatomical Structure,SCT,11068009,T8 vertebra,,,,...,"[Bone structure of T8, Bone structure of T8 (b...","[Eighth thoracic vertebra, Eighth dorsal verte...",[],[],"[Body structure (body structure), Body structure]",[Body structures],"[Bone structure of T8, Bone structure of T8 (b...","[Eighth thoracic vertebra, Eighth dorsal verte...",[],[]
100,vertebrae_T9,SCT,123037004,Anatomical Structure,SCT,82687006,T9 vertebra,,,,...

In [16]:
df.to_csv('slicerMappingsToSnomedMappings.csv')

In [17]:
df=df[['Structure',
       'SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator',
       'SegmentedPropertyCategoryCodeSequence.CodeValue',
       'SegmentedPropertyCategoryCodeSequence.CodeMeaning',
       'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
       'SegmentedPropertyTypeCodeSequence.CodeValue',
       'SegmentedPropertyTypeCodeSequence.CodeMeaning',
       'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
       'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
       'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning']]
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,adrenal_gland_left,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,7771000,Left
1,adrenal_gland_right,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,24028007,Right
2,aorta,SCT,123037004,Anatomical Structure,SCT,15825003,Aorta,,,
3,autochthon_left,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,7771000,Left
4,autochthon_right,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,24028007,Right
...,...,...,...,...,...,...,...,...,...,...
99,vertebrae_T8,SCT,123037004,Anatomical Structure,SCT,11068009,T8 vertebra,,,
100,vertebrae_T9,SCT,123037004,Anatomical Structure,SCT,82687006,T9 vertebra,,,
101,vertebrae_T10,SCT,123037004,Anatomical Structure,SCT,7610001,T10 vertebra,,,
102,vertebrae_T11,SCT,123037004,Anatomical Structure,SCT,12989004,T11 vertebra,,,


In [18]:
df.to_csv('TotalSegmentator v1 SCT mapping - total.csv')

###**Trying to use Slicer's json config (only has 87 parts)**

In [19]:
try:
  os.remove(f'{curr_dir}/SegmentationCategoryTypeModifier-TotalSegmentator.term.json')
except OSError:
  pass
!wget https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/Resources/SegmentationCategoryTypeModifier-TotalSegmentator.term.json




--2023-10-04 09:53:04--  https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/Resources/SegmentationCategoryTypeModifier-TotalSegmentator.term.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55864 (55K) [text/plain]
Saving to: ‘SegmentationCategoryTypeModifier-TotalSegmentator.term.json’

SegmentationCategor 100%[===================>]  54.55K  --.-KB/s    in 0.007s  

2023-10-04 09:53:04 (7.43 MB/s) - ‘SegmentationCategoryTypeModifier-TotalSegmentator.term.json’ saved [55864/55864]



In [20]:
import json
import pandas as pd

with open('SegmentationCategoryTypeModifier-TotalSegmentator.term.json', 'r') as file:
    json_data = json.load(file)

rows = []
for category in json_data['SegmentationCodes']['Category']:
    for item in category['Type']:
        row = {}
        row['CodeValue'] = item['CodeValue']
        row['CodeMeaning'] = item['CodeMeaning']
        row['CodingSchemeDesignator'] = item['CodingSchemeDesignator']
        if 'Modifier' in item:
            for modifier in item['Modifier']:
                new_row = row.copy()
                new_row['ModifierCodeValue'] = modifier['CodeValue']
                new_row['ModifierCodeMeaning'] = modifier['CodeMeaning']
                new_row['ModifierCodingSchemeDesignator'] = modifier['CodingSchemeDesignator']
                if '3dSlicerLabel' in modifier:
                    new_row['3dSlicerLabel'] = modifier['3dSlicerLabel']
                if 'recommendedDisplayRGBValue' in modifier:
                    new_row['recommendedDisplayRGBValue'] = modifier['recommendedDisplayRGBValue']
                rows.append(new_row)
        else:
            rows.append(row)

df = pd.DataFrame(rows)

# Rename the columns to match the desired column names
df = df.rename(columns={
    'CodingSchemeDesignator': 'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
    'CodeValue': 'SegmentedPropertyTypeCodeSequence.CodeValue',
    'CodeMeaning': 'SegmentedPropertyTypeCodeSequence.CodeMeaning',
    'ModifierCodingSchemeDesignator': 'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
    'ModifierCodeValue': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
    'ModifierCodeMeaning': 'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'
})
filtered_df = df[pd.notna(df['3dSlicerLabel'])]
filtered_df

,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,3dSlicerLabel,recommendedDisplayRGBValue
1,79601000,Scapula,SCT,24028007,Right,SCT,right scapula,"[212, 188, 102]"
2,79601000,Scapula,SCT,7771000,Left,SCT,left scapula,"[212, 188, 102]"
4,85050009,Humerus,SCT,24028007,Right,SCT,right humerus,"[205, 179, 108]"
5,85050009,Humerus,SCT,7771000,Left,SCT,left humerus,"[205, 179, 108]"
7,71341001,Femur,SCT,24028007,Right,SCT,right femur,"[255, 238, 170]"
8,71341001,Femur,SCT,7771000,Left,SCT,left femur,"[255, 238, 170]"
10,44947003,Erector spinae muscle,SCT,24028007,Right,SCT,right erector spinae muscle,"[171, 85, 68]"
11,44947003,Erector spinae muscle,SCT,7771000,Left,SCT,left erector spinae muscle,"[171, 85, 68]"
16,73634005,Common iliac artery,SCT,24028007,Right,SCT,right common iliac artery,"[216, 101, 79]"
17,73634005,Common iliac artery,SCT,7771000,Left,SCT,left common iliac artery,"[216, 101, 79]"
